In [ ]:
import sys
sys.path.append('..')
import configs.config as cfg
import pandas as pd
import numpy as np

# Loading single cell stats and transformed matrix
df = pd.read_csv(f"/mnt/c/Users/trang.le/Desktop/shapemode/S-BIAD34/shapemodes/{cfg.ALIGNMENT}_{cfg.MODE}/transformed_matrix.csv")
df = df.drop('Unnamed: 0', axis=1)
df['Protein_nu_mean'] = df['Protein_nu_sum']/df['nu_area']
df['Protein_cyt_mean'] = (df['Protein_cell_sum']- df['Protein_nu_sum'])/(df['cell_area']- df['nu_area'])
df['CDT1_nu_mean'] = df['CDT1_nu_sum']/df['nu_area']
#df['Protein_cell_mean'] = df['Protein_cell_sum']/df['cell_area']
df['MT_cell_mean'] = df['MT_cell_sum']/df['cell_area']
df['CDT1_nu_mean'] = df['CDT1_nu_sum']/df['nu_area']
df['Protein_cyt_mean'] = (df['Protein_cell_sum']- df['Protein_nu_sum'])/(df['cell_area']- df['nu_area'])

In [ ]:
# Meta data
mappings = pd.read_csv(f"/mnt/c/Users/trang.le/Desktop/shapemode/S-BIAD34/experimentB-processed.txt", sep="\t")
ifimages = pd.read_csv(f"/mnt/c/Users/trang.le/Downloads/IF-image.csv")
ifimages = ifimages[ifimages.atlas_name=='U2OS']
ab_loc = ifimages[['antibody','locations']].drop_duplicates()
mappings = mappings.merge(ab_loc, left_on='Antibody id', right_on='antibody')

In [ ]:
PERMUTATIONS = 10000
mv_window = 20

def get_r2_numpy_corrcoef(x, y):
    return np.corrcoef(x, y)[0, 1]**2

def mvavg(yvals, mv_window):
    '''Calculate the moving average'''
    return np.convolve(yvals, np.ones((mv_window,))/mv_window, mode='valid')

import scipy.stats as st
def mvavg_ci(yvals, window_size, ci = 0.95):
    '''Calculate the moving confidence interval 
    '''
    min_max = []
    for i in range(len(yvals) - window_size + 1):
        a = yvals[i: i + window_size]
        min_max += [st.t.interval(ci, len(a)-1, loc=np.mean(a), scale=st.sem(a))]
    return np.array(min_max)

def plot_moving_averages(df,ab, PC, feature_name):
    df_ = df[df.ab_id==ab]
    sorted_indices = np.argsort(df_[PC])
    # Sort data along PCx
    sorted_feature1 = df_[feature_name].values[sorted_indices.values]
    sorted_pos = df_[PC].values[sorted_indices.values]
    sorted_mt = df_['MT_cell_mean'].values[sorted_indices.values]
    sorted_speudotime = df_['pseudotime'].values[sorted_indices.values]
    
    # Normalize/standardize
    sorted_feature1 = np.log10(sorted_feature1)
    sorted_mt = np.log10(sorted_mt)
    
    # Apply moving average
    sorted_mt_mvavg = mvavg(sorted_mt, mv_window)
    sorted_feature1_mvavg = mvavg(sorted_feature1, mv_window)
    sorted_pc_mvavg = mvavg(sorted_pos, mv_window)
    # Plots
    plt.figure()
    plt.scatter(sorted_pos, sorted_feature1, color='blue', alpha=0.1, label='protein intensity')
    plt.plot(sorted_pc_mvavg, sorted_feature1_mvavg, c='blue')
    plt.scatter(sorted_pos, sorted_mt, color='grey', alpha=0.1, label='MT intensity')
    plt.plot(sorted_pc_mvavg, sorted_mt_mvavg, c='grey')
    #plt.plot(sorted_pc_mvavg, sorted_feature2_mvavg, c='red')
    #mvavg_min = pd.Series(sorted_feature1).rolling(mv_window).min()[mv_window-1:]
    #mvavg_max = pd.Series(sorted_feature1).rolling(mv_window).max()[mv_window-1:]
    ci_minmax = mvavg_ci(sorted_feature1, mv_window, ci = 0.95)
    plt.vlines(sorted_pc_mvavg, ci_minmax[:,0], ci_minmax[:,1], color="blue", alpha=0.2)
    plt.title(f'{ab}: {PC} vs {feature_name}')
    plt.legend()

In [ ]:
feature1 = 'Protein_nu_mean'
feature2 = 'Protein_cyt_mean'
results = []
for pc in [1]: #range(1,7):
    PC = f'PC{pc}'
    for ab_, df_ in df.groupby('ab_id'):
        sorted_indices = np.argsort(df_[PC])
        # Sort data along PCx
        sorted_feature1 = df_[feature1].values[sorted_indices.values]
        sorted_feature2 = df_[feature2].values[sorted_indices.values]
        sorted_pos = df_[PC].values[sorted_indices.values]
        sorted_mt = df_['MT_cell_mean'].values[sorted_indices.values]
        sorted_speudotime = df_['pseudotime'].values[sorted_indices.values]

        # Normalize/standardize
        sorted_feature1 = np.log10(sorted_feature1)
        sorted_feature2 = np.log10(sorted_feature2)
        sorted_mt = np.log10(sorted_mt)

        # Apply moving average
        sorted_mt_mvavg = mvavg(sorted_mt, mv_window)
        sorted_feature1_mvavg = mvavg(sorted_feature1, mv_window)
        sorted_feature2_mvavg = mvavg(sorted_feature2, mv_window)
        sorted_pc_mvavg = mvavg(sorted_pos, mv_window)

        # Permutations
        perms = [np.random.permutation(len(df_[feature1].values)) for _ in range(10000)]
        features = np.log10(df_[feature1].values)
        # Metric : mean difference from random
        curr_rng_comp = [features[perm] for perm in perms]
        curr_mvavg_rng_comp = [mvavg(rng_feats, mv_window) for rng_feats in curr_rng_comp]
        pervar_ = np.var(sorted_feature1_mvavg)/np.var(features)
        pervar_ordered = np.var(curr_mvavg_rng_comp,axis=1) / np.var(features)
        mean_diff_1 = np.mean(pervar_ - pervar_ordered)
        
        
        # Permutations
        perms = [np.random.permutation(len(df_[feature2].values)) for _ in range(10000)]
        features = np.log10(df_[feature2].values)
        # Metric : mean difference from random
        curr_rng_comp = [features[perm] for perm in perms]
        curr_mvavg_rng_comp = [mvavg(rng_feats, mv_window) for rng_feats in curr_rng_comp]
        pervar_ = np.var(sorted_feature2_mvavg)/np.var(features)
        pervar_ordered = np.var(curr_mvavg_rng_comp,axis=1) / np.var(features)
        mean_diff_2 = np.mean(pervar_ - pervar_ordered)
        results += [[ab_,PC, mean_diff_1, mean_diff_2]]
        #results[ab_] = {PC: {'mean_diff_nu': mean_diff_1, 'mean_diff_cyt': mean_diff_2}}
results = pd.DataFrame(results)
results.columns = ['ab','PC','mean_diff_nu','mean_diff_cyt']

In [ ]:
results[results.mean_diff_nu>=0.09].sort_values('mean_diff_nu')

In [ ]:
import matplotlib.pyplot as plt
plot_moving_averages(df,'HPA060948','PC1','Protein_nu_mean')
plot_moving_averages(df,'HPA060948','PC1','Protein_cyt_mean')
mappings[mappings.antibody=='HPA060948']